# Performance Metrics

Sharpe ratio, Sortino ratio, Calmar ratio, and summary statistics for the WolfpackTrend strategy.

**Data Source:**
- `wolfpack/daily_snapshots.csv` - Daily NAV for return calculation

**Analysis:**
- Daily return series from NAV
- Sharpe ratio (daily and annualized)
- Sortino ratio (downside risk adjusted)
- Calmar ratio (return/max drawdown)
- Configurable risk-free rate

**Prerequisites:** Run the WolfpackTrend backtest first to generate ObjectStore data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

from QuantConnect import *
from QuantConnect.Research import QuantBook

qb = QuantBook()
print("QuantBook initialized")

## Configuration

In [ ]:
# Risk-free rate (annualized)
# Set to 0.0 for zero risk-free rate, or actual rate like 0.05 for 5%
RISK_FREE_RATE = 0.0

# Trading days per year
TRADING_DAYS = 252

print(f"Risk-free rate: {RISK_FREE_RATE * 100:.2f}%")
print(f"Trading days per year: {TRADING_DAYS}")

## Load Data

In [ ]:
try:
    snapshots_str = qb.ObjectStore.Read("wolfpack/daily_snapshots.csv")
    df = pd.read_csv(StringIO(snapshots_str))
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').reset_index(drop=True)
    
    print(f"Loaded {len(df)} daily snapshots")
    print(f"Date range: {df['date'].min().strftime('%Y-%m-%d')} to {df['date'].max().strftime('%Y-%m-%d')}")
    
    if 'nav' not in df.columns:
        raise ValueError("NAV column not found in daily_snapshots.csv")
        
except Exception as e:
    print(f"ERROR: {e}")
    print("Make sure you have run the WolfpackTrend backtest first.")
    df = pd.DataFrame()

## Compute Daily Returns

In [ ]:
if not df.empty:
    # Compute daily returns from NAV
    df['daily_return'] = df['nav'].pct_change()
    
    # Daily risk-free rate
    daily_rf = RISK_FREE_RATE / TRADING_DAYS
    df['excess_return'] = df['daily_return'] - daily_rf
    
    # Drop first row (NaN return)
    returns = df['daily_return'].dropna()
    excess_returns = df['excess_return'].dropna()
    
    print(f"\nDaily Returns:")
    print("=" * 60)
    print(f"Count: {len(returns)}")
    print(f"Mean: {returns.mean() * 100:.4f}%")
    print(f"Std: {returns.std() * 100:.4f}%")
    print(f"Min: {returns.min() * 100:.4f}%")
    print(f"Max: {returns.max() * 100:.4f}%")

## Helper Functions

In [ ]:
def sharpe_ratio(returns, risk_free_rate=0.0, periods_per_year=252):
    """
    Calculate annualized Sharpe ratio.
    
    Args:
        returns: Series of daily returns
        risk_free_rate: Annual risk-free rate
        periods_per_year: Number of periods per year
    
    Returns:
        Annualized Sharpe ratio
    """
    daily_rf = risk_free_rate / periods_per_year
    excess_returns = returns - daily_rf
    
    if excess_returns.std() == 0:
        return np.nan
    
    return (excess_returns.mean() / excess_returns.std()) * np.sqrt(periods_per_year)


def sortino_ratio(returns, risk_free_rate=0.0, periods_per_year=252):
    """
    Calculate annualized Sortino ratio (uses downside deviation).
    
    Args:
        returns: Series of daily returns
        risk_free_rate: Annual risk-free rate
        periods_per_year: Number of periods per year
    
    Returns:
        Annualized Sortino ratio
    """
    daily_rf = risk_free_rate / periods_per_year
    excess_returns = returns - daily_rf
    
    # Downside deviation (only negative returns)
    downside_returns = excess_returns[excess_returns < 0]
    downside_std = np.sqrt((downside_returns ** 2).mean())
    
    if downside_std == 0:
        return np.nan
    
    return (excess_returns.mean() / downside_std) * np.sqrt(periods_per_year)


def calmar_ratio(returns, periods_per_year=252):
    """
    Calculate Calmar ratio (annualized return / max drawdown).
    
    Args:
        returns: Series of daily returns
        periods_per_year: Number of periods per year
    
    Returns:
        Calmar ratio
    """
    # Annualized return
    cumulative = (1 + returns).cumprod()
    total_return = cumulative.iloc[-1] - 1
    years = len(returns) / periods_per_year
    annualized_return = (1 + total_return) ** (1 / years) - 1
    
    # Max drawdown
    running_max = cumulative.cummax()
    drawdown = (cumulative / running_max) - 1
    max_drawdown = abs(drawdown.min())
    
    if max_drawdown == 0:
        return np.nan
    
    return annualized_return / max_drawdown


def max_drawdown(returns):
    """
    Calculate maximum drawdown.
    
    Args:
        returns: Series of daily returns
    
    Returns:
        Maximum drawdown (positive number)
    """
    cumulative = (1 + returns).cumprod()
    running_max = cumulative.cummax()
    drawdown = (cumulative / running_max) - 1
    return abs(drawdown.min())


print("Helper functions defined")

## Performance Summary

In [ ]:
if not df.empty:
    # Calculate metrics
    total_return = (df['nav'].iloc[-1] / df['nav'].iloc[0]) - 1
    years = len(returns) / TRADING_DAYS
    annualized_return = (1 + total_return) ** (1 / years) - 1
    annualized_vol = returns.std() * np.sqrt(TRADING_DAYS)
    
    sharpe = sharpe_ratio(returns, RISK_FREE_RATE, TRADING_DAYS)
    sortino = sortino_ratio(returns, RISK_FREE_RATE, TRADING_DAYS)
    calmar = calmar_ratio(returns, TRADING_DAYS)
    max_dd = max_drawdown(returns)
    
    print("\n" + "=" * 80)
    print("PERFORMANCE SUMMARY")
    print("=" * 80)
    
    print(f"\nPeriod: {df['date'].min().strftime('%Y-%m-%d')} to {df['date'].max().strftime('%Y-%m-%d')}")
    print(f"Trading days: {len(returns)}")
    print(f"Years: {years:.2f}")
    
    print(f"\nReturn Metrics:")
    print(f"  Total Return: {total_return * 100:.2f}%")
    print(f"  Annualized Return: {annualized_return * 100:.2f}%")
    print(f"  Annualized Volatility: {annualized_vol * 100:.2f}%")
    
    print(f"\nRisk-Adjusted Metrics (Risk-Free Rate: {RISK_FREE_RATE * 100:.2f}%):")
    print(f"  Sharpe Ratio: {sharpe:.4f}")
    print(f"  Sortino Ratio: {sortino:.4f}")
    print(f"  Calmar Ratio: {calmar:.4f}")
    
    print(f"\nDrawdown:")
    print(f"  Maximum Drawdown: {max_dd * 100:.2f}%")
    
    print("\n" + "=" * 80)

## Cumulative Performance

In [ ]:
if not df.empty:
    df['cumulative_return'] = (1 + df['daily_return'].fillna(0)).cumprod()
    df['running_max'] = df['cumulative_return'].cummax()
    df['drawdown'] = (df['cumulative_return'] / df['running_max']) - 1
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
    
    # Cumulative returns
    axes[0].plot(df['date'], (df['cumulative_return'] - 1) * 100, linewidth=2, color='steelblue', label='Strategy')
    axes[0].fill_between(df['date'], 0, (df['cumulative_return'] - 1) * 100, alpha=0.3, color='steelblue')
    axes[0].set_title('Cumulative Return', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Cumulative Return (%)')
    axes[0].axhline(y=0, color='black', linestyle='-', alpha=0.3)
    axes[0].legend(loc='upper left')
    axes[0].grid(True, alpha=0.3)
    
    # Drawdown
    axes[1].fill_between(df['date'], 0, df['drawdown'] * 100, color='red', alpha=0.5)
    axes[1].plot(df['date'], df['drawdown'] * 100, linewidth=1, color='darkred')
    axes[1].set_title('Underwater Plot (Drawdown)', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Date')
    axes[1].set_ylabel('Drawdown (%)')
    axes[1].grid(True, alpha=0.3)
    
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

## Rolling Performance Metrics

In [ ]:
if not df.empty:
    # Rolling Sharpe (252-day)
    window = 252
    
    rolling_mean = returns.rolling(window).mean()
    rolling_std = returns.rolling(window).std()
    daily_rf = RISK_FREE_RATE / TRADING_DAYS
    rolling_sharpe = ((rolling_mean - daily_rf) / rolling_std) * np.sqrt(TRADING_DAYS)
    
    # Rolling Sortino
    def rolling_sortino_calc(x):
        excess = x - daily_rf
        downside = excess[excess < 0]
        if len(downside) == 0:
            return np.nan
        downside_std = np.sqrt((downside ** 2).mean())
        if downside_std == 0:
            return np.nan
        return (excess.mean() / downside_std) * np.sqrt(TRADING_DAYS)
    
    rolling_sortino = returns.rolling(window).apply(rolling_sortino_calc, raw=False)
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
    
    # Rolling Sharpe
    axes[0].plot(df['date'].iloc[1:], rolling_sharpe, linewidth=2, color='steelblue')
    axes[0].axhline(y=0, color='red', linestyle='--', alpha=0.5)
    axes[0].axhline(y=1, color='green', linestyle='--', alpha=0.5, label='Sharpe = 1')
    axes[0].set_title(f'Rolling {window}-Day Sharpe Ratio', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Sharpe Ratio')
    axes[0].legend(loc='upper left')
    axes[0].grid(True, alpha=0.3)
    
    # Rolling Sortino
    axes[1].plot(df['date'].iloc[1:], rolling_sortino, linewidth=2, color='coral')
    axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.5)
    axes[1].axhline(y=1, color='green', linestyle='--', alpha=0.5, label='Sortino = 1')
    axes[1].set_title(f'Rolling {window}-Day Sortino Ratio', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Date')
    axes[1].set_ylabel('Sortino Ratio')
    axes[1].legend(loc='upper left')
    axes[1].grid(True, alpha=0.3)
    
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

## Monthly Returns

In [ ]:
if not df.empty:
    # Group by month
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    
    # Calculate monthly returns
    monthly = df.groupby(['year', 'month']).agg({
        'nav': ['first', 'last']
    })
    monthly.columns = ['nav_start', 'nav_end']
    monthly['monthly_return'] = (monthly['nav_end'] / monthly['nav_start']) - 1
    monthly = monthly.reset_index()
    
    # Create heatmap data
    heatmap_data = monthly.pivot(index='year', columns='month', values='monthly_return') * 100
    heatmap_data.columns = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                            'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'][:len(heatmap_data.columns)]
    
    # Plot heatmap
    fig, ax = plt.subplots(figsize=(12, max(4, len(heatmap_data) * 0.8)))
    
    sns.heatmap(heatmap_data, annot=True, fmt='.1f', cmap='RdYlGn', center=0,
                linewidths=1, ax=ax, cbar_kws={'label': 'Return (%)'})
    
    ax.set_title('Monthly Returns Heatmap (%)', fontsize=14, fontweight='bold')
    ax.set_xlabel('Month')
    ax.set_ylabel('Year')
    
    plt.tight_layout()
    plt.show()
    
    # Monthly statistics
    print("\nMonthly Return Statistics:")
    print("=" * 60)
    print(f"Best month: {monthly['monthly_return'].max() * 100:.2f}%")
    print(f"Worst month: {monthly['monthly_return'].min() * 100:.2f}%")
    print(f"Average month: {monthly['monthly_return'].mean() * 100:.2f}%")
    print(f"Positive months: {(monthly['monthly_return'] > 0).sum()} / {len(monthly)} ({(monthly['monthly_return'] > 0).mean() * 100:.1f}%)")

## Return Distribution

In [ ]:
if not df.empty:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Daily return histogram
    axes[0].hist(returns * 100, bins=50, color='steelblue', alpha=0.7, edgecolor='black')
    axes[0].axvline(x=0, color='red', linestyle='--', linewidth=2)
    axes[0].axvline(x=returns.mean() * 100, color='green', linestyle='--', linewidth=2, 
                    label=f'Mean: {returns.mean() * 100:.4f}%')
    axes[0].set_title('Daily Return Distribution', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Daily Return (%)')
    axes[0].set_ylabel('Frequency')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Monthly return histogram
    axes[1].hist(monthly['monthly_return'] * 100, bins=20, color='coral', alpha=0.7, edgecolor='black')
    axes[1].axvline(x=0, color='red', linestyle='--', linewidth=2)
    axes[1].axvline(x=monthly['monthly_return'].mean() * 100, color='green', linestyle='--', linewidth=2,
                    label=f'Mean: {monthly["monthly_return"].mean() * 100:.2f}%')
    axes[1].set_title('Monthly Return Distribution', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Monthly Return (%)')
    axes[1].set_ylabel('Frequency')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Distribution statistics
    print("\nReturn Distribution Statistics:")
    print("=" * 60)
    print(f"Daily:")
    print(f"  Skewness: {returns.skew():.4f}")
    print(f"  Kurtosis: {returns.kurtosis():.4f}")
    print(f"  Positive days: {(returns > 0).sum()} / {len(returns)} ({(returns > 0).mean() * 100:.1f}%)")
    print(f"\nMonthly:")
    print(f"  Skewness: {monthly['monthly_return'].skew():.4f}")
    print(f"  Kurtosis: {monthly['monthly_return'].kurtosis():.4f}")

## Performance Table

In [ ]:
if not df.empty:
    # Create comprehensive metrics table
    metrics = {
        'Metric': [
            'Total Return',
            'Annualized Return',
            'Annualized Volatility',
            'Sharpe Ratio',
            'Sortino Ratio',
            'Calmar Ratio',
            'Maximum Drawdown',
            'Best Day',
            'Worst Day',
            'Best Month',
            'Worst Month',
            'Win Rate (Daily)',
            'Win Rate (Monthly)',
            'Avg Win / Avg Loss (Daily)',
            'Skewness',
            'Kurtosis'
        ],
        'Value': [
            f"{total_return * 100:.2f}%",
            f"{annualized_return * 100:.2f}%",
            f"{annualized_vol * 100:.2f}%",
            f"{sharpe:.4f}",
            f"{sortino:.4f}",
            f"{calmar:.4f}",
            f"{max_dd * 100:.2f}%",
            f"{returns.max() * 100:.4f}%",
            f"{returns.min() * 100:.4f}%",
            f"{monthly['monthly_return'].max() * 100:.2f}%",
            f"{monthly['monthly_return'].min() * 100:.2f}%",
            f"{(returns > 0).mean() * 100:.1f}%",
            f"{(monthly['monthly_return'] > 0).mean() * 100:.1f}%",
            f"{abs(returns[returns > 0].mean() / returns[returns < 0].mean()):.2f}",
            f"{returns.skew():.4f}",
            f"{returns.kurtosis():.4f}"
        ]
    }
    
    metrics_df = pd.DataFrame(metrics)
    
    print("\n" + "=" * 60)
    print("COMPREHENSIVE PERFORMANCE METRICS")
    print("=" * 60)
    display(metrics_df)